In [ ]:
## Athenian JupyterLab Environment

Welcome to the Athenian JupyterLab environment. This is a dedicated machine for your own analysis over Athenian API data. This environment gives you secure way to analyze data from the Athenian API. 

You can find the documentation to our API [here](https://api.athenian.co/v1/ui/#).

In [19]:
%load_ext dotenv

In [21]:
%dotenv

In [27]:
import os
ATHENIAN_TOKEN=os.getenv("ATHENIAN_TOKEN")

In [24]:
import json
import requests
import pandas as pd
from scipy import stats
import numpy as np
import re

ACCOUNT = 28

date_from = "2021-04-01" 
date_to = "2021-05-24"

headers = {
    'X-API-Key': ATHENIAN_TOKEN,
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
headers

{'X-API-Key': 'CiQAxPAFJnO1cCARdOIfycxA3kANCl8zC5koMEhL3ZJtkxakXSYSMQCmdPTXhK1/5pm/saB2VCurio+LAz80+3uuY3ixVWzkkEuZLAcOUtb4jntfMucyZCo=',
 'accept': 'application/json',
 'Content-Type': 'application/json'}

## Retrieving the teams

Let's retrieve the organization's teams

In [26]:
r = requests.get("https://api.athenian.co/v1/teams/{ACCOUNT}",
                 headers=headers)
r
# teams = r.json()
# summary = []

# for team in teams:
#     summary.append([team["id"], team["name"], len(team["members"])])
    
# df_teams = pd.DataFrame(summary)
# df_teams.columns = ['id', 'name', '# of members']
# df_teams
# r

<Response [400]>

## Retrieving Jira users mapped to GitHub contributors

In [83]:
def map_devs_to_jira(contribs):
    response = requests.get(f"https://api.athenian.co/v1/settings/jira/identities/{ACCOUNT}",
                     headers=headers)

    jira_mapped = []

    i = 0
    for jira_user in response.json():
        if jira_user["developer_id"] is None:
            continue
        if jira_user["developer_id"] in contribs and jira_user["confidence"] >= 0.5:
            i+=1
            jira_mapped.append(jira_user["jira_name"])
            
    return jira_mapped
 
jira_users = map_devs_to_jira(contribs)
    
print(f"{len(jira_users)} out of {len(contribs)} contributors were mapped to their Jira account.\n\nYou can increase this number by manually mapping users in Athenian > Settings > Teams")    


56 out of 59 contributors were mapped to their Jira account.

You can increase this number by manually mapping users in Athenian > Settings > Teams


## Retrieving different ticket types used

In [5]:
data = {
        "account":28,
        "date_from": date_from,
        "date_to": date_to,
        "exclude_inactive":True,
        "return":["issues","issue_types"],
#         "with": {"assignees": jira_users,
#                  "reporters":[],
#                  "commenters":[]}
       }

r = requests.post('https://api.athenian.co/v1/filter/jira', headers=headers, data=json.dumps(data))
results = r.json()

summary_issue_types = []

for result in results["issue_types"]:
    if result["name"] not in summary_issue_types:
        summary_issue_types.append(result["name"])
        
print(summary_issue_types)

['Bug', 'Change', 'Cloud', 'Contribution', 'Documentation', 'Epic', 'Guidelines', 'Help Center', 'Idea', 'Improvement', 'Mock-ups', 'New Feature', 'Proofreading', 'QA', 'Question', 'Service Request', 'Story', 'Sub-task', 'Subtask', 'Task', 'Technical task', 'UX']


## Analyzing support tickets

You can see the different metrics available on tickets [here](https://api.athenian.co/v1/ui/#/metrics/calc_metrics_jira_linear)

In [85]:
# Small helper function to convert second strings to hours

def str_to_hours(st): 
    s = int(st.replace('s',''))
    h = round(s / 60 / 60, 1)
    return h

In [97]:
# Please set the ticket types here you'd like to include
ticket_types = ["Bug"]

metrics = ["jira-open",
            "jira-resolved", 
            "jira-raised",
            "jira-life-time", 
            "jira-lead-time",
            "jira-acknowledge-time",
            "jira-resolution-rate"]


data = {"account": ACCOUNT,
        "date_from": date_from,
        "date_to": date_to,
        "exclude_inactive":True,
        "metrics": metrics,
        "granularities":["aligned month"],
        "timezone":0,
        "quantiles":[0,1],
        "types": ticket_types,
        "priorities":[],
        "labels_include":[],
        # You can comment out the "with" parameter to retrieve these metrics over all tickets
        "with":
            [{"assignees": jira_users,             
              "reporters": jira_users,
              "commenters":[],
             }],
        "group_by_jira_label":False}

r = requests.post('https://api.athenian.co/v1/metrics/jira', headers=headers, data=json.dumps(data))

# A great way to see the data is to print the JSON and view it in http://jsonviewer.stack.hu/
# 
# print(r.text)

results = r.json()

rows = []
data = []

for month in results[0]["values"]:
    rows.append(month["date"])
    data.append([month["values"][0],
          month["values"][1],
          month["values"][2],
          month["values"][3],
          month["values"][4],
          month["values"][5],
          month["values"][6]])

df_jira_metrics = pd.DataFrame(data, rows, metrics)
# convert from string in seconds to float of hours
df_jira_metrics[["jira-life-time", "jira-lead-time", "jira-acknowledge-time"]] = df_jira_metrics[["jira-life-time", "jira-lead-time", "jira-acknowledge-time"]].applymap(str_to_hours)
df_jira_metrics

,jira-open,jira-resolved,jira-raised,jira-life-time,jira-lead-time,jira-acknowledge-time,jira-resolution-rate
2021-04-01,146,266,239,485.0,320.2,182.3,1.112500
2021-05-01,114,192,160,1544.6,1461.5,115.6,1.198758


## Maintainance ticket Analysis (May 24th, 2021)

In [164]:
data = {
        "account": ACCOUNT,
        "date_from": date_from,
        "date_to": date_to,
        "exclude_inactive":True,
        "types": ["Bug"],
        "return":["issues", "statuses"],
#         "with": {"assignees": jira_users,
#                  "reporters":jira_users,
#                  "commenters":jira_users}
       }

r = requests.post('https://api.athenian.co/v1/filter/jira', headers=headers, data=json.dumps(data))
results = r.json()

for status in results["statuses"]:
    if status["project"] == "10000":
        print(status["name"], status["stage"])

Approved In Progress
Closed Done
In Progress In Progress
Needs approval In Progress
Open To Do
Resolved Done


In [165]:
# Small helper function to convert second strings to hours

def str_to_hours(st): 
    s = int(st.replace('s',''))
    h = round(s / 60 / 60, 1)
    return h

In [166]:
def filter_bug_tickets_project(project_id, statuses_input):

    data = {
            "account": ACCOUNT,
            "date_from": date_from,
            "date_to": date_to,
            "exclude_inactive":True,
            "types": ["Bug"],
            "return":["issues", "issue_bodies"],
    #         "with": {"assignees": jira_users,
    #                  "reporters":jira_users,
    #                  "commenters":jira_users}
           }

    r = requests.post('https://api.athenian.co/v1/filter/jira', headers=headers, data=json.dumps(data))
    results = r.json()

    filtered_issues = []
    statuses = []

    for result in results["issues"]:
        if int(result["project"]) == project_id:

            if result["status"] not in statuses_input:
                continue

            matches = re.findall('\[(.*?)\]', result["title"] )
                            
            filtered_issues.append([result["title"], 
                                    matches, 
                                    result["created"],
                                    result["updated"],
                                    result["work_began"],
                                    str_to_hours(result["lead_time"]),
                                    str_to_hours(result["life_time"]),
                                    result["status"],
                                    result["priority"]
                                   ])



    print(len(filtered_issues))

    titles = ["title", "title_labels", "created", "updated", "work_began", "lead_time", "life_time", "status", "priority"]

    tickets_df = pd.DataFrame(filtered_issues)
    tickets_df.columns = titles

    return tickets_df

df1 = filter_bug_tickets_project(10000, ["Closed", "Resolved"])

df1

81


,title,title_labels,created,updated,work_began,lead_time,life_time,status,priority
0,[SLA] Grid breaks when ref entity record used ...,[SLA],2021-04-20T12:28:01.319000000,2021-04-23T07:42:01.683000000,2021-04-20T12:50:48.237000000,50.7,51.1,Closed,Major
1,[SLA] SVG files are not saved on a ref entity ...,[SLA],2021-04-21T09:02:06.006000000,2021-04-27T12:30:34.398000000,2021-04-21T09:18:10.801000000,797.6,797.9,Closed,Major
2,[SLA] [Serenity] Scroll in reference entity at...,"[SLA, Serenity]",2021-04-21T11:58:02.374000000,2021-04-22T10:12:22.559000000,2021-04-21T13:03:03.398000000,49.6,50.7,Closed,Major
3,[Serenity] Error 500 on the product grid with ...,[Serenity],2021-04-21T16:40:27.223000000,2021-04-27T08:16:18.435000000,2021-04-22T12:20:56.138000000,770.6,790.2,Closed,Major
4,[CRITICAL] Asset Manager: Error 500 after del...,[CRITICAL],2021-04-22T12:03:25.325000000,2021-05-03T09:36:11.000000000,2021-04-26T07:50:45.175000000,232.2,324.0,Closed,Major
...,...,...,...,...,...,...,...,...,...
76,[SLA] Backport PIM-9705 to 5.0,[SLA],2021-04-14T12:37:21.656000000,2021-04-15T14:47:28.226000000,2021-04-14T14:59:09.586000000,23.8,26.1,Closed,Major
77,[SLA] [Serenity]Translation strings missing fo...,"[SLA, Serenity]",2021-04-15T08:27:32.754000000,2021-04-15T09:44:03.145000000,2021-04-15T08:32:44.769000000,4.7,4.8,Closed,Minor
78,[SLA] Infinite loop on Asset API,[SLA],2021-04-15T16:14:32.752000000,2021-04-20T12:44:19.293000000,2021-04-16T09:26:40.194000000,99.3,116.5,Closed,Major
79,[SLA] Reference Entity Import XLSX fails if im...,[SLA],2021-04-15T16:36:25.194000000,2021-04-22T07:12:47.315000000,2021-04-16T08:03:02.076000000,143.1,158.6,Closed,Minor


In [170]:
def filter_fn(row):
    return all(x in row['title_labels'] for x in ["Serenity", "SLA"])

m = df1.apply(filter_fn, axis=1)

filtered_df = df1[m]
filtered_df

,title,title_labels,created,updated,work_began,lead_time,life_time,status,priority
2,[SLA] [Serenity] Scroll in reference entity at...,"[SLA, Serenity]",2021-04-21T11:58:02.374000000,2021-04-22T10:12:22.559000000,2021-04-21T13:03:03.398000000,49.6,50.7,Closed,Major
16,[SLA] [Serenity] MySQL error when trying to im...,"[SLA, Serenity]",2021-04-28T21:34:18.295000000,2021-05-17T07:20:22.603000000,2021-04-29T07:31:05.610000000,387.3,397.3,Closed,Major
17,[SLA] [Serenity] The options page of attribute...,"[SLA, Serenity]",2021-04-28T21:43:24.531000000,2021-05-17T07:20:31.904000000,2021-05-04T10:37:37.150000000,484.3,617.2,Closed,Major
39,[SLA] [Serenity] Reference Entity import fails...,"[SLA, Serenity]",2021-05-17T22:50:54.310000000,2021-05-24T07:32:34.663000000,2021-05-18T12:47:02.276000000,146.1,160.1,Resolved,Major
40,[SLA] [Serenity] Logs can't be downloaded anym...,"[SLA, Serenity]",2021-05-19T07:26:59.787000000,2021-05-21T11:16:43.097000000,2021-05-19T07:52:12.683000000,127.0,127.5,Closed,Major
41,[SLA] [Serenity] The Columns button of the Pub...,"[SLA, Serenity]",2021-05-19T22:59:47.582000000,2021-05-24T13:18:22.728000000,2021-05-20T09:18:51.889000000,101.6,111.9,Resolved,Major
56,[SLA][Serenity] Optimize Query in Class SqlGet...,"[SLA, Serenity]",2021-03-27T01:25:56.280000000,2021-04-05T12:45:57.389000000,2021-03-29T15:18:37.860000000,1343.6,1405.5,Closed,Major
58,"[SLA] [Serenity] The ""Unit locale"" dropdown fo...","[SLA, Serenity]",2021-03-29T16:15:32.778000000,2021-04-02T09:02:34.632000000,2021-03-30T07:15:21.119000000,77.4,92.3,Closed,Major
64,[SLA] [Serenity] Completeness doesn't update o...,"[SLA, Serenity]",2021-04-05T23:40:44.801000000,2021-04-07T16:31:22.575000000,2021-04-06T16:50:45.000000000,1150.1,1167.2,Closed,Major
66,[SLA] [Serenity] Event not sent when creating ...,"[SLA, Serenity]",2021-04-07T12:46:35.895000000,2021-04-13T05:49:31.659000000,2021-04-07T13:13:59.800000000,1129.7,1130.1,Closed,Major


In [172]:
lead_time_mean_hrs = round(filtered_df["lead_time"].mean())

print(f"Average lead time for filtered_df: {lead_time_mean_hrs}")

Average lead time for filtered_df: 500


In [178]:
grouped_df = filtered_df.groupby("priority")
mean_df = grouped_df["lead_time"].mean().round(0)
mean_df = mean_df.reset_index()
mean_df

,priority,lead_time
0,Major,500.0
1,Minor,498.0
